There exist mentions of elibility requirements in page text, some in specialized `Eligibility` sections, others within the context of `smart_answers` or `answer` document_types. With a focus on age requirements, extract stuff out.

In [503]:
import os
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from pprint import pprint
import spacy
from lxml import etree
from bs4 import BeautifulSoup
from dateutil.parser import parse

In [374]:
from textblob import TextBlob

In [25]:
DATA_DIR = os.getenv("DATA_DIR")
content_path = os.path.join(DATA_DIR, 
                            "preprocessed_content_store_wdetails_june_en_june.csv.gz")
df = pd.read_csv(content_path, compression="gzip")

/Users/felisialoukou/.virtualenvs/k-graph/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
df.head()

,base_path,content_id,title,description,document_type,details,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale
0,/aaib-reports/aaib-investigation-to-hawker-sea...,96eacfbe-0385-45ef-9289-8428dacad258,"AAIB investigation to Hawker Sea Fury T MK 20,...","Engine failure and landing gear collapse, RNAS...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The aircraft was performing in a publ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
1,/aaib-reports/aaib-investigation-to-hph-glasfl...,1d697c99-b1d0-4855-b72d-a97d83a4fc91,"AAIB investigation to HPH Glasflugel 304 eS, G...",Front Electric Sustainer (FES) battery fire du...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: During a normal touchdown following a...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
2,/aaib-reports/aaib-investigation-to-ikarus-c42...,5814334a-77d0-426e-8e78-ff2b05ea6322,"AAIB investigation to Ikarus C42 FB UK, G-IKUS\t",Aircraft crashed whilst avoiding a hedge when ...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot was attempting to take off ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
3,/aaib-reports/aaib-investigation-to-ikarus-c42...,c8f31c76-eab0-4be6-95a5-5e6e7f32056c,"AAIB investigation to Ikarus C42 FB100, G-CEHG\t","Overturned on landing, Farm Strip, Hardwicke, ...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: G-CEHG was landing on a private airst...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
4,/aaib-reports/aaib-investigation-to-ikarus-c42...,4cc0ae15-ad87-42ce-8a89-2bdb14e19e26,"AAIB investigation to Ikarus C42 FB100, G-ZAVI","Aircraft struck sheep on landing, Lundy Island...",aaib_report,{'metadata': {'date_of_occurrence': '2014-06-2...,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot reported that this was his ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en


In [27]:
df['details'] = df['details'].map(literal_eval)

In [28]:
def flatten_dict(parts_dict):
    return {item['slug']:"".join([b['content'] 
                                  for b in item['body']]) for item in parts_dict}    

In [29]:
df['details_parts'] = df['details'].map(lambda x: flatten_dict(x['parts']) 
                                        if 'parts' in x.keys() 
                                          else np.nan)

In [30]:
df['eligibility_html'] = df['details_parts'].map(lambda x: x.get('eligibility',np.nan) 
                                                 if not isinstance(x,float) else np.nan)

In [31]:
df_welig = df[~df['eligibility_html'].isna()]

In [37]:
def extract_text(body):
    """
    Extract text from html body
    :param body: <str> containing html.
    """
    # TODO: Tidy this up!
    r = None
    # body != "\n" and
    if body and body != "\n" and not body.isspace():
        try:
            # print("this is", body)
            tree = etree.HTML(body)
            r = tree.xpath('//text()')
            r = ' '.join(r)
            r = r.strip().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
            r = r.replace('\n', ' ').replace(',', ' ')
            # r = r.lower()
            r = ' '.join(r.split())
        except ValueError:
            print("exception @ extract:", type(body), body)
    if not r:
        r = ' '
    return r

In [55]:
df_welig.base_path.values

array(['/1619-bursary-fund', '/additional-state-pension',
       '/access-to-elected-office-fund', '/access-to-work',
       '/attendance-allowance', '/become-an-mot-tester',
       '/bereavement-allowance', '/bereavement-payment',
       '/bereavement-support-payment', '/blind-persons-allowance',
       '/agricultural-sick-pay', '/budgeting-help-benefits',
       '/asylum-support', '/adult-dependants-grant',
       '/adoption-pay-leave', '/advanced-learner-loan', '/ancestry-visa',
       '/apply-citizenship-born-uk', '/apply-citizenship-british-parent',
       '/apply-for-a-uk-residence-card', '/care-to-learn',
       '/carers-allowance', '/carers-credit', '/childcare-grant',
       '/christmas-bonus', '/claim-asylum',
       '/claim-compensation-criminal-injury', '/child-benefit',
       '/child-maintenance', '/child-study-visa',
       '/claim-for-injury-received-while-serving',
       '/cold-weather-payment', '/constant-attendance-allowance',
       '/derivative-right-residence', '

In [46]:
### BeautifulSoup attempt to get all the bulleted lists/headers out

In [62]:
test = df_welig.iloc[0]
text = test.eligibility_html
soup = BeautifulSoup(text, "html.parser")

In [63]:
test.base_path

'/1619-bursary-fund'

In [484]:
text[0:10]

'You can on'

    if header -> new scenario
    first header (overall?)
    other headers -> secondary scenarios, unless first header mentions a requirement disputed by scenario, then it's a different class

In [108]:
def specialized_scenario(header_string):
    return any([x in header_string for x in [' for ','group']]) 

In [109]:
specialized_scenario("group")

True

In [501]:
class Section:
    
    def __init__(self, section_type="", header=""):
        self.section_type = section_type
        self.header = header
        self.bullets = []
        self.paragraphs = []
    
    def set_type(self, section_type):
        self.section_type = section_type
        
    def set_header(self, header):
        self.header = header
    
    def set_list_header(self, paragraph_text):
        self.bullets.append([paragraph_text,
                             self._is_exclusion(paragraph_text),
                             []])
        
    def add_bullet(self, bullet):
        if bullet not in self.bullets[-1][2]:
            self.bullets[-1][2].append(bullet)
            
    def add_paragraph(self, paragraph):
        if paragraph not in self.paragraphs:
            self.paragraphs.append(paragraph)   
            
    def to_string(self):
        return "type: {}\nheader: {}\nbullets: {}\nparagraphs: {}"\
                    .format(self.section_type, self.header, self.bullets, self.paragraphs)
    
    def to_dict(self):
        return {'type': self.section_type, 
                'header': self.header,
                'bullets': self.bullets,
                'paragraphs': self.paragraphs}
    
    def _is_exclusion(self,text):
        return bool(re.search("not (eligible|qualify)|can('t|not)\s?(get|apply)?", text)) or \
                        TextBlob(text).sentiment.polarity < -0.2
    
    def _contains_date(self, text, fuzzy=False):
        try: 
            parse(text, fuzzy=fuzzy)
            return True

        except ValueError:
            return False

In [508]:
import datefinder

matches = datefinder.find_dates("You could get up to £1,200 if at least one of the following")

for match in matches:
    print(match)

0200-01-26 00:00:00


In [500]:
Section._contains_date(_,"you were over under 18 on August 2019")

False

In [502]:
Section._is_exclusion(_,"You could get up to £1,200 if at least one of the following applies:")
TextBlob("You could get up to £1,200 if at least one of the following applies:").sentiment.polarity

-0.15

In [471]:
samples = ['''You’ll usually get less than the full amount, or no bursary, '''
            '''if one of the following apply''',
          'you must', 
           'If you’re not automatically a citizen, you may be eligible to apply to ‘register’ as one.',
          'you are not eligible if', 
           'Who is not eligible',
           'Who can’t get Care to Learn',
           'You won’t be paid for the first 3 days off sick unless you’re away for longer than 14 working days in total.']
for s in samples:
    print(s)
    print(TextBlob(s).sentiment.polarity)
    print(is_neg(s))
    print("###")

You’ll usually get less than the full amount, or no bursary, if one of the following apply
-0.016666666666666663
False
###
you must
0.0
False
###
If you’re not automatically a citizen, you may be eligible to apply to ‘register’ as one.
0.0
False
###
you are not eligible if
0.0
True
###
Who is not eligible
0.0
True
###
Who can’t get Care to Learn
0.0
True
###
You won’t be paid for the first 3 days off sick unless you’re away for longer than 14 working days in total.
-0.15476190476190477
False
###


In [457]:
section = Section()
section.set_list_header("you must")
section.add_bullet("womp")
section.set_list_header("wat")
section.add_bullet("womp")
section.bullets

[['you must', False, ['womp']], ['wat', False, ['womp']]]

In [311]:
            
#         if header.name = "ul":
# #             print(header.name, header.text.strip())
#         elif header.name == "ul":
#             for bullet in header.find_all(["li"]):
# #                 print("+", bullet.text.strip())
#         else:
#             print("bonk")
#         if "h" in header.name and specialized_scenario(header.text.strip()):
#             print('https://www.gov.uk'+row['base_path']+'/eligibility')
#             print("secondary case?")
#             print(header.name, header.text.strip())
#         print("###")

In [312]:
df_welig[0:2]

,base_path,content_id,title,description,document_type,details,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale,details_parts,eligibility_html
20,/1619-bursary-fund,f4b96a38-5247-4afd-b554-8a258a0e8c93,16 to 19 Bursary Fund,"Bursaries of up to £1,200 for students in furt...",guide,"{'parts': [{'title': 'Overview', 'slug': 'over...",{'organisations': ['71381a6e-aa5c-43ae-a982-be...,{'organisations': ['Education and Skills Fundi...,NaN,NaN,You could get a bursary to help with education...,"[{'title': 'Education, training and skills', '...",en,{'overview': 'You could get a bursary to help ...,You must:\r\n\r\n+ be at least 16 and under 19...
8732,/additional-state-pension,e78637eb-3be4-408c-9f9c-d2336635c0ca,Additional State Pension,"Additional State Pension, also known as the St...",guide,"{'parts': [{'title': 'Overview', 'slug': 'over...",{'organisations': ['b548a09f-8b35-4104-89f4-f1...,{'organisations': ['Department for Work and Pe...,NaN,NaN,The Additional State Pension is an extra amoun...,"[{'title': 'Work', 'content_id': 'd0f1e5a3-c8f...",en,{'overview': 'The Additional State Pension is ...,##You reached State Pension age on or after 6 ...


In [458]:
def get_type(header):
    if header=="p":
        return "base"
    return "secondary"

In [459]:
all_stacks = []
for i,row in df_welig.iterrows():
    text = row['eligibility_html']
    soup = BeautifulSoup(text, "html.parser")
    section_stack = []
    print("https://www.gov.uk"+row['base_path']+"/eligibility")
    items = list(soup.find_all(["h1","h2","h3", "p", "ul"]))
    
    # first element on eligibility page
    section_stack = [Section(get_type(items[0].name), items[0].text.strip())]
    if len(items)>1 and items[1].name == "ul":
        section_stack[-1].set_list_header(items[0].text.strip()) 
        
    for j, header in enumerate(items[1:],1):
        print(f'at index: {j} type: <{header.name}>\nwords: {header.text}')
        if header.name.startswith("h"):
            section_stack.append(Section("secondary",header.text.strip()))
            if j+1< len(items) and items[j+1].name == "ul":
                section_stack[-1].set_list_header(header.text.strip())    

        elif header.name == "p":
            if j+1< len(items) and items[j+1].name == "ul":
                section_stack[-1].set_list_header(header.text.strip())  
            else:
                 section_stack[-1].add_paragraph(header.text.strip())
        else:
            for bullet in header.find_all(["li"]):
                section_stack[-1].add_bullet(bullet.text.strip())
    print("####")
    all_stacks.append([row['base_path'],section_stack])

https://www.gov.uk/1619-bursary-fund/eligibility
at index: 1 type: <ul>
words: 
be at least 16 and under 19 on 31 August 2019
study at a publicly funded school or college, or be on an unpaid training course
meet the residency requirements - your school or college can check this

at index: 2 type: <h2>
words: Bursary for students in vulnerable groups
at index: 3 type: <p>
words: You could get up to £1,200 if at least one of the following applies:
at index: 4 type: <ul>
words: 
you’re in or recently left local authority care
you get Income Support or Universal Credit because you’re financially supporting yourself
you get Disability Living Allowance (DLA) in your name and either Employment and Support Allowance (ESA) or Universal Credit
you get Personal Independence Payment (PIP) in your name and either ESA or Universal Credit

at index: 5 type: <p>
words: You may get the full amount if you have expenses and study full-time on a course of at least 30 weeks.
at index: 6 type: <p>
words: Yo

words: Child Benefit tax charge
at index: 26 type: <p>
words: You’ll still be eligible for Child Benefit even if you choose to stop receiving it because you or your partner has an income over £50,000. Stopping your Child Benefit payments does not affect your entitlement - you can always change your mind and restart them.
at index: 27 type: <p>
words: Contact the Child Benefit Office if you’re not sure about your eligibility.
####
https://www.gov.uk/child-maintenance/eligibility
at index: 1 type: <ul>
words: 
under 16
under 20 and in full-time education (but not higher than A Level or equivalent)
under 20 and living with a parent who has registered for Child Benefit for them

at index: 2 type: <p>
words: There are also other situations when child maintenance payments stop.
at index: 3 type: <h2>
words: Who can use the Child Maintenance Service
at index: 4 type: <p>
words: You can apply if you’re:
at index: 5 type: <ul>
words: 
the parent the child lives with
the parent the child does no


at index: 22 type: <p>
words: You’ll also need to show you’re looking for work to keep getting payments.
at index: 23 type: <p>
words: Your partner’s income and savings will not affect your claim.
at index: 24 type: <p>
words: You can get contribution-based JSA for up to 182 days (about 6 months). After this you can talk to your work coach about your options.
at index: 25 type: <h2>
words: Income-based JSA

at index: 26 type: <p>
words: Your employment in the last 2 to 3 years does not affect your eligibility.
at index: 27 type: <p>
words: You can only apply if you either:
at index: 28 type: <ul>
words: 
get the severe disability premium, or are entitled to it
got or were entitled to the severe disability premium within the last month and are still eligible for it

at index: 29 type: <p>
words: You’ll also need to:
at index: 30 type: <ul>
words: 
be 18 or over (there are some exceptions if you’re 16 or 17 - contact Jobcentre Plus for advice)
be under the State Pension age

not be in f

at index: 1 type: <ul>
words: 
Income Support
income-based Jobseeker’s Allowance (JSA)

income-related Employment and Support Allowance (ESA)

Universal Credit
Pension Credit

at index: 2 type: <p>
words: You can start getting a loan:
at index: 3 type: <ul>
words: 
from the date you start getting Pension Credit
after you’ve had 9 consecutive Universal Credit payments
after you’ve claimed any other qualifying benefit for 39 consecutive weeks

at index: 4 type: <p>
words: You might still be able to get SMI if you apply for one of the qualifying benefits but cannot get it because your income is too high. You’ll then be treated as getting the benefit you applied for.
at index: 5 type: <p>
words: You will not be treated as getting Universal Credit if you cannot get it because your income is too high.
at index: 6 type: <p>
words: Contact the relevant office to check if you’re eligible for SMI.
####
https://www.gov.uk/standard-visitor-visa/eligibility
at index: 1 type: <ul>
words: 
you’ll lea

In [460]:
all_stacks[0]

['/1619-bursary-fund',

In [539]:
text = "this happened obefore April 2019"


'before April 2019'

In [560]:
df_welig.reset_index(drop=True,inplace=True)

In [568]:
cond = r"(on|before|by|after|between|from|until|in|up to).*\s\d{4}"
counter = 0
for i,row in df_welig.iterrows():
    text = row['eligibility_html']
    one = bool(re.search(cond, text))
#     two =  bool(re.search(".*\d{4}.*", text))
# not one and two
    if one:
        print(i,":",row['base_path'])
        print("first:",re.search(cond, text).group(0))
        print([date.strftime("%d/%m/%Y")
               for date in datefinder.find_dates(re.search(cond, text).group(0))])
        counter+=1
#         print("second:",re.search(".*\d{4}.*", text).group(0))
        print("####")
counter, df_welig.shape

0 : /1619-bursary-fund
first: on 31 August 2019
['31/08/2019']
####
1 : /additional-state-pension
first: on age on or after 6 April 2016
['06/04/2016']
####
6 : /bereavement-allowance
first: before 6 April 2017
['06/04/2017']
####
7 : /bereavement-payment
first: before 6 April 2017
['06/04/2017']
####
8 : /bereavement-support-payment
first: on or after 6 April 2017
['06/04/2017']
####
10 : /agricultural-sick-pay
first: only have the right to Agricultural Sick Pay if you were employed before the [rules changed on 1 October 2013
['01/10/2013']
####
15 : /advanced-learner-loan
first: before 1 August 2016
['01/08/2016']
####
16 : /ancestry-visa
first: before 31 March 1922
['31/03/1922']
####
17 : /apply-citizenship-born-uk
first: on or after 1 January 1983
['01/01/1983']
####
18 : /apply-citizenship-british-parent
first: on or after 1 July 2006
['01/07/2006']
####
20 : /care-to-learn
first: ing in the UK until 30 June 2021, or 31 December 2020
['31/12/2020']
####
22 : /carers-credit
first:

(42, (127, 15))

In [461]:
for thing, section in all_stacks:
    print(thing)
    for sec in section:
        print(sec.to_string())
        print("###")
    print("###\n")

/1619-bursary-fund
type: base
header: You must:
bullets: [['You must:', False, ['be at least 16 and under 19 on 31 August 2019', 'study at a publicly funded school or college, or be on an unpaid training course', 'meet the residency requirements - your school or college can check this']]]
paragraphs: []
###
type: secondary
header: Bursary for students in vulnerable groups
bullets: [['You could get up to £1,200 if at least one of the following applies:', True, ['you’re in or recently left local authority care', 'you get Income Support or Universal Credit because you’re financially supporting yourself', 'you get Disability Living Allowance (DLA) in your name and either Employment and Support Allowance (ESA) or Universal Credit', 'you get Personal Independence Payment (PIP) in your name and either ESA or Universal Credit']], ['You’ll usually get less than the full amount, or no bursary, if one of the following apply:', True, ['your course is shorter than 30 weeks', 'you study part time', 

/vat-margin-schemes
type: base
header: You can only use a margin scheme for:
bullets: [['You can only use a margin scheme for:', False, ['second-hand goods', 'works of art', 'antiques and collectors’ items']]]
paragraphs: []
###
type: secondary
header: Second-hand goods
bullets: []
paragraphs: ['Goods that can still be used, or which could be used after repair.']
###
type: secondary
header: Works of art
bullets: []
paragraphs: ['Most items normally described as ‘works of art’ are eligible. There are some exceptions, eg technical drawings, scenery for theatres and hand-decorated manufactured items.']
###
type: secondary
header: Antiques and collectors’ items
bullets: []
paragraphs: ['Antiques are goods that are over 100 years old. Collectors’ items are stamps, coins and currency and other pieces of scientific, historical or archaeological interest. Not all items that can be collected are eligible for a margin scheme.', 'There are special rules for works of art, antiques or collectors’ i

In [315]:
### Regex stuff

In [ ]:
rowlist = []
for key,value in list(elig_texts.items()):

    if "This fund is currently closed." not in value:
#         \s?\d\d\s?(and|or)?\s?(over|under)?(\s\d\d)?
        search = re.finditer(""".*(born on|(reached|over|under) State Pension age|
                             age(d)?|('re|were|are|be( at least)?|have been)\s?(over|under)?\d\d).*""", value)
        if search is not None:
#             print(f'https://www.gov.uk{key}/eligibility')
            for i,s in enumerate(search):
                start = s.start()
                end = s.end()
#                 print(f'At {i}: ({start},{end})')
                text = value[start:end]
#                 print(text)
                rowlist.append({'base_path':key, 
                                'instance #':int(i+1),
                                'start':start,
                                'end':end,
                                'age_text':value[start:end],
                                'text':value})
        else:
            rowlist.append({'base_path':key, 'age_text':'Age requirements not detected',
                           'text':value})
            
    else:
#         print(f'https://www.gov.uk{key}/eligibility')
#         print("closed")
        rowlist.append({'base_path':key, 'age_text':'Fund closed','text':value})
df_eligibility = pd.DataFrame(rowlist)
df_eligibility = df_eligibility[['base_path', 'text','instance #', 'start', 'end','age_text']]